In [1]:
 %tensorflow_version 1.x 


import keras.backend as K
import tensorflow as tf

from keras.engine.topology import InputSpec
from keras.engine.topology import Layer

from keras.utils import Sequence

from keras.layers import concatenate
from keras.layers import BatchNormalization, GlobalAveragePooling2D, Reshape
from keras.layers import Conv2D, Activation, Input, MaxPooling2D, ZeroPadding2D

from keras.models import Model

from keras.regularizers import l2

from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import SGD, Adam

import numpy as np
import cv2
import random

import pandas as pd
import matplotlib.pyplot as plt


class Conv2DNormalization(Layer):
    def __init__(self, scale, **kwargs):
        self.axis = 3
        self.scale = scale
        super(Conv2DNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (input_shape[self.axis],)
        init_gamma = self.scale * np.ones(shape)
        self.gamma = K.variable(init_gamma, name='{}_gamma'.format(self.name))
        self.trainable_weights = [self.gamma]

    def call(self, x, mask=None):
        output = K.l2_normalize(x, self.axis)
        output = output * self.gamma
        return output

Using TensorFlow backend.


In [ ]:
def SSD(input_shape=(300, 300, 3),
        n_classes=3,
        n_boxes=[4, 6, 6, 6, 4, 4],
        weights=None,
        l2_regularization=0.0005):
    """
    Define the SSD model

    :param input_shape: Input image shape
    :param n_classes: number of output classes (object types)
    :param n_boxes: number of predefined anchor boxes in each SSD feature extractor layer
    :param l2_regularizaion: Regularization term hyperparameter
    """
    
    x = Input(shape=input_shape)

    ##### Start - base model - Vgg16 #####
    conv1_1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv1_1')(x)
    conv1_2 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv1_2')(conv1_1)
    pool1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name='pool1')(conv1_2)


    conv2_1 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv2_1')(pool1)
    conv2_2 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv2_2')(conv2_1)
    pool2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name='pool2')(conv2_2)


    conv3_1 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv3_1')(pool2)
    conv3_2 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv3_2')(conv3_1)
    conv3_3 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv3_3')(conv3_2)
    pool3 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name='pool3')(conv3_3)


    conv4_1 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv4_1')(pool3)
    conv4_2 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv4_2')(conv4_1)
    conv4_3 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv4_3')(conv4_2)
    conv4_3_norm = Conv2DNormalization(20, name='conv4_3_norm')(conv4_3)
    pool4 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name='pool4')(conv4_3)



    conv5_1 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv5_1')(pool4)
    conv5_2 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv5_2')(conv5_1)
    conv5_3 = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv5_3')(conv5_2)
    pool5 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same', name='pool5')(conv5_3)    
    ##### End - base model - Vgg16 #####
    
    fc6 = Conv2D(1024, (3, 3), dilation_rate=(6, 6), activation='relu', padding='same', kernel_initializer='he_normal',
                 kernel_regularizer=l2(l2_regularization), name='fc6')(pool5)
    fc7 = Conv2D(1024, (1, 1), activation='relu', padding='same', kernel_initializer='he_normal',
                 kernel_regularizer=l2(l2_regularization), name='fc7')(fc6)
    

    conv6_1 = Conv2D(256, (1, 1), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv6_1')(fc7)
    conv6_1 = ZeroPadding2D(padding=((1, 1), (1, 1)), name='conv6_padding')(conv6_1)
    conv6_2 = Conv2D(512, (3, 3), strides=(2, 2), activation='relu', padding='valid', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv6_2')(conv6_1)


    conv7_1 = Conv2D(128, (1, 1), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv7_1')(conv6_2)
    conv7_1 = ZeroPadding2D(padding=((1, 1), (1, 1)), name='conv7_padding')(conv7_1)
    conv7_2 = Conv2D(256, (3, 3), strides=(2, 2), activation='relu', padding='valid', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv7_2')(conv7_1)


    conv8_1 = Conv2D(128, (1, 1), activation='relu', padding='same', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv8_1')(conv7_2)
    conv8_2 = Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='valid', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv8_2')(conv8_1)


    conv9_1 = Conv2D(128, (1, 1), activation='relu', padding='same', kernel_initializer='he_normal', 
                     kernel_regularizer=l2(l2_regularization), name='conv9_1')(conv8_2)
    conv9_2 = Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='valid', kernel_initializer='he_normal',
                     kernel_regularizer=l2(l2_regularization), name='conv9_2')(conv9_1)


    # confidence predictions
    conv4_3_norm_conf = Conv2D(n_boxes[0] * n_classes, (3, 3), padding='same', kernel_initializer='he_normal',
                               kernel_regularizer=l2(l2_regularization), name='conv4_3_norm_conf')(conv4_3_norm)
    fc7_conf = Conv2D(n_boxes[1] * n_classes, (3, 3), padding='same', kernel_initializer='he_normal',
                               kernel_regularizer=l2(l2_regularization), name='fc7_conf')(fc7)
    conv6_2_conf = Conv2D(n_boxes[2] * n_classes, (3, 3), padding='same', kernel_initializer='he_normal',
                               kernel_regularizer=l2(l2_regularization), name='conv6_2_conf')(conv6_2)
    conv7_2_conf = Conv2D(n_boxes[3] * n_classes, (3, 3), padding='same', kernel_initializer='he_normal',
                               kernel_regularizer=l2(l2_regularization), name='conv7_2_conf')(conv7_2)
    conv8_2_conf = Conv2D(n_boxes[4] * n_classes, (3, 3), padding='same', kernel_initializer='he_normal',
                               kernel_regularizer=l2(l2_regularization), name='conv8_2_conf')(conv8_2)
    conv9_2_conf = Conv2D(n_boxes[5] * n_classes, (3, 3), padding='same', kernel_initializer='he_normal',
                               kernel_regularizer=l2(l2_regularization), name='conv9_2_conf')(conv9_2)
    

    # location predictions
    conv4_3_norm_loc = Conv2D(n_boxes[0] * 4, (3, 3), padding='same', kernel_initializer='he_normal',
                               kernel_regularizer=l2(l2_regularization), name='conv4_3_norm_loc')(conv4_3_norm)
    fc7_loc = Conv2D(n_boxes[1] * 4, (3, 3), padding='same', kernel_initializer='he_normal',
                               kernel_regularizer=l2(l2_regularization), name='fc7_loc')(fc7)
    conv6_2_loc = Conv2D(n_boxes[2] * 4, (3, 3), padding='same', kernel_initializer='he_normal',
                               kernel_regularizer=l2(l2_regularization), name='conv6_2_loc')(conv6_2)
    conv7_2_loc = Conv2D(n_boxes[3] * 4, (3, 3), padding='same', kernel_initializer='he_normal',
                               kernel_regularizer=l2(l2_regularization), name='conv7_2_loc')(conv7_2)
    conv8_2_loc = Conv2D(n_boxes[4] * 4, (3, 3), padding='same', kernel_initializer='he_normal',
                               kernel_regularizer=l2(l2_regularization), name='conv8_2_loc')(conv8_2)
    conv9_2_loc = Conv2D(n_boxes[5] * 4, (3, 3), padding='same', kernel_initializer='he_normal',
                               kernel_regularizer=l2(l2_regularization), name='conv9_2_loc')(conv9_2)

    # Flatten non-class axises of confidence predictions
    conv4_3_norm_conf_reshape = Reshape((-1, n_classes), name='conv4_3_norm_conf_reshape')(conv4_3_norm_conf)
    fc7_conf_reshape = Reshape((-1, n_classes), name='fc7_conf_reshape')(fc7_conf)
    conv6_2_conf_reshape = Reshape((-1, n_classes), name='conv6_2_conf_reshape')(conv6_2_conf)
    conv7_2_conf_reshape = Reshape((-1, n_classes), name='conv7_2_conf_reshape')(conv7_2_conf)
    conv8_2_conf_reshape = Reshape((-1, n_classes), name='conv8_2_conf_reshape')(conv8_2_conf)
    conv9_2_conf_reshape = Reshape((-1, n_classes), name='conv9_2_conf_reshape')(conv9_2_conf)

    # Flatten non-location axises of location predictions
    conv4_3_norm_loc_reshape = Reshape((-1, 4), name='conv4_3_norm_loc_reshape')(conv4_3_norm_loc)
    fc7_loc_reshape = Reshape((-1, 4), name='fc7_loc_reshape')(fc7_loc)
    conv6_2_loc_reshape = Reshape((-1, 4), name='conv6_2_loc_reshape')(conv6_2_loc)
    conv7_2_loc_reshape = Reshape((-1, 4), name='conv7_2_loc_reshape')(conv7_2_loc)
    conv8_2_loc_reshape = Reshape((-1, 4), name='conv8_2_loc_reshape')(conv8_2_loc)
    conv9_2_loc_reshape = Reshape((-1, 4), name='conv9_2_loc_reshape')(conv9_2_loc)

    # Concatenate non-class axises
    conf = concatenate([conv4_3_norm_conf_reshape,
                             fc7_conf_reshape,
                             conv6_2_conf_reshape,
                             conv7_2_conf_reshape,
                             conv8_2_conf_reshape,
                             conv9_2_conf_reshape], axis=1, name='conf')

    # Concatenate non-location axises
    loc = concatenate([conv4_3_norm_loc_reshape,
                            fc7_loc_reshape,
                            conv6_2_loc_reshape,
                            conv7_2_loc_reshape,
                            conv8_2_loc_reshape,
                            conv9_2_loc_reshape], axis=1, name='loc')

    conf_softmax = Activation('softmax', name='conf_softmax')(conf)
    predictions = concatenate([loc, conf_softmax], axis=2, name='predictions')


    model = Model(inputs=x, outputs=predictions)

    if weights is not None:
        model.load_weights(weights, by_name=True)

    return model

In [ ]:
class MultiboxLoss(object):
    def __init__(self, num_classes, neg_pos_ratio=3,
                 alpha=1.0, background_id=0, max_num_negatives=300):
        self.num_classes = num_classes
        self.alpha = alpha
        self.neg_pos_ratio = neg_pos_ratio
        self.background_id = background_id
        self.max_num_negatives = max_num_negatives

    def smooth_l1(self, y_true, y_pred):
        absolute_value_loss = K.abs(y_true - y_pred) - 0.5
        square_loss = 0.5 * (y_true - y_pred)**2
        absolute_value_condition = K.less(absolute_value_loss, 1.0)
        l1_smooth_loss = tf.where(absolute_value_condition, square_loss,
                                  absolute_value_loss)
        return K.sum(l1_smooth_loss, axis=-1)

    def cross_entropy(self, y_true, y_pred):
        y_pred = K.maximum(K.minimum(y_pred, 1 - 1e-15), 1e-15)
        cross_entropy_loss = - K.sum(y_true * K.log(y_pred), axis=-1)
        return cross_entropy_loss

    def __call__(self, y_true, y_pred):
        class_loss = self.cross_entropy(y_true[:, :, 4:], y_pred[:, :, 4:])
        local_loss = self.smooth_l1(y_true[:, :, :4], y_pred[:, :, :4])
        negative_mask = y_true[:, :, 4 + self.background_id]
        positive_mask = 1 - negative_mask

        positive_local_losses = local_loss * positive_mask
        positive_class_losses = class_loss * positive_mask
        positive_class_loss = K.sum(positive_class_losses, axis=-1)
        positive_local_loss = K.sum(positive_local_losses, axis=-1)
        
        num_positives_per_sample = K.cast(K.sum(positive_mask, -1), 'int32')
        num_hard_negatives = self.neg_pos_ratio * num_positives_per_sample
        num_negatives_per_sample = K.minimum(num_hard_negatives,
                                             self.max_num_negatives)
        negative_class_losses = class_loss * negative_mask

        elements = (negative_class_losses, num_negatives_per_sample)
        negative_class_loss = tf.map_fn(
                lambda x: K.sum(tf.nn.top_k(x[0], x[1])[0]),
                elements, dtype=tf.float32)

        class_loss = positive_class_loss + negative_class_loss
        total_loss = class_loss + (self.alpha * positive_local_loss)

        batch_mask = K.not_equal(num_positives_per_sample, 0)
        total_loss = tf.where(batch_mask, total_loss, K.zeros_like(total_loss))

        num_positives_per_sample = tf.where(
                batch_mask, num_positives_per_sample,
                tf.cast(K.ones_like(num_positives_per_sample), tf.int32))

        num_positives_per_sample = K.cast(num_positives_per_sample, 'float32')
        total_loss = total_loss / num_positives_per_sample
        return total_loss

In [4]:
import keras
vgg = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', 
                                     input_tensor=None, input_shape=(300, 300, 3), pooling=None, classes=1000)
vgg.save_weights('weights_vgg.hdf5')

In [ ]:
from itertools import product
def precompute_priors(image_size=300, feature_map_sizes=[38, 19, 10, 5, 3, 1], steps=[8, 16, 32, 64, 100, 300],
                      min_sizes=[30, 60, 111, 162, 213, 264], max_sizes=[60, 111, 162, 213, 264, 315], 
                      aspect_ratios=[[2], [2, 3], [2, 3], [2, 3], [2], [2]], variance=[0.1, 0.2], point_form=False):
    boxes = []
    for idx, map_size in enumerate(feature_map_sizes):
        for x in range(map_size):
            for y in range(map_size):
                f_k = image_size / steps[idx]
                cx = (x + 0.5) / f_k
                cy = (y + 0.5) / f_k
                s_k = min_sizes[idx] / image_size
                boxes = boxes + [cx, cy, s_k, s_k]
                s_k_prime = np.sqrt(s_k * (max_sizes[idx] / image_size))
                boxes = boxes + [cx, cy, s_k_prime, s_k_prime]
                for aspect_ratio in aspect_ratios[idx]:
                    boxes = boxes + [cx, cy, s_k * np.sqrt(aspect_ratio), s_k / np.sqrt(aspect_ratio)]
                    boxes = boxes + [cx, cy, s_k / np.sqrt(aspect_ratio), s_k * np.sqrt(aspect_ratio)]

    boxes = np.asarray(boxes).reshape((-1, 4))
    boxes = np.clip(boxes, 0, 1)

    if point_form:
        return np.concatenate((boxes[:, :2] - boxes[:, 2:]/2, boxes[:, :2] + boxes[:, 2:]/2), 1)

    return boxes

In [6]:
!unzip drive/My\ Drive/IUST-DIP/data.zip -d data/
!7z x data/dataset.7z

Archive:  drive/My Drive/IUST-DIP/data.zip
replace data/dataset.7z? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/dataset.7z         
  inflating: data/detections.xlsx    
  inflating: data/ground-truth.xlsx  

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 67541227 bytes (65 MiB)

Extracting archive: data/dataset.7z
--
Path = data/dataset.7z
Type = 7z
Physical Size = 67541227
Headers Size = 7942
Method = LZMA2:24
Solid = +
Blocks = 1

    
Would you like to replace the existing file:
  Path:     ./dataset/train/004.txt
  Size:     0 bytes
  Modified: 2019-12-19 11:46:42
with the file from archive:
  Path:     dataset/train/004.txt
  Size:     0 bytes
  Modified: 2019-12-19 11:46:42
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? A

  0% 3 - dataset/

In [ ]:
class DataManager(object):
    def __init__(self, dataset_path='dataset', train_val_path=None, mode='train', num_classes=None):
        if num_classes is None:
            self.num_classes = 2
        self.class_keys = np.arange(2)+1  # [1, 2]
        self.dataset_path = dataset_path
        if train_val_path is None:
            self.train_val_path = ['train', 'valid']
        self.train_data = dict()  # {'image_name': array(size=(num_objects, 4 + num_classes)}
        self.test_data = dict()
        self.mode = mode

    def _load_data(self):
        names = self._load_names()
        for name in names:
            with open(name + '.txt', 'r') as f:
                txt = f.readlines()
            num_objects = len(txt)
            
            bounding_boxes = []
            one_hot_classes = []
            if num_objects == 0:
                annotation = np.zeros((1, 4 + self.num_classes))
                if self.mode == 'train':
                    self.train_data[name+'.jpg'] = annotation
                    continue
                elif self.mode == 'test':
                    self.test_data[name+'.jpg'] = annotation
                    continue
                else:
                    raise Exception('wrong mode!')
                
            for l in txt:
                coor_class = l.split(' ')
                bounding_boxes.append([int(float(coor)) for coor in coor_class[:-1]])
                one_hot_class = [0] * (self.num_classes  + 1)  # add background class with 0 value
                one_hot_class[int(float(coor_class[-1]))] = 1
                one_hot_classes.append(one_hot_class)
            
            bounding_boxes = np.asarray(bounding_boxes)
            one_hot_classes = np.asarray(one_hot_classes)
            annotation = np.hstack((bounding_boxes, one_hot_classes))
            if self.mode == 'train':
                self.train_data[name+'.jpg'] = annotation
            elif self.mode == 'test':
                self.test_data[name+'.jpg'] = annotation
            else:
                raise Exception('wrong mode!')
        if self.mode == 'train':
            return self.train_data
        elif self.mode == 'test':
            return self.test_data
        else:
            raise Exception('wrong mode!')
        

    def _load_names(self):
        import glob
        names = []
        if self.mode == 'train':
            path = self.train_val_path[0]
        elif self.mode == 'test':
            path = self.train_val_path[1]
        else:
            raise Exception('Wrong mode!')

        for file in glob.glob(self.dataset_path+'/'+path+'/'+'*.jpg'):
            names.append(file[:-4]) 
        return names

    def __call__(self):
        return self._load_data()

In [8]:
train_data = DataManager()()
print(list(train_data.keys())[1])
print(list(train_data.values())[1])

test_data = DataManager(mode='test')()
print(list(test_data.keys())[1])
print(list(test_data.values())[1])

dataset/train/100.jpg
[[0. 0. 0. 0. 0. 0.]]
dataset/valid/597.jpg
[[608  49 682 108   0   1   0]]


In [ ]:
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, boxes=None, labels=None):
        for t in self.transforms:
            img, boxes, labels = t(img, boxes, labels)
        return img, boxes, labels


class ToFloat(object):
    def __init__(self, d_type=np.float32):
        self.d_type = d_type
    def __call__(self, image, boxes=None, labels=None):
        return image.astype(self.d_type), boxes.astype(self.d_type), labels.astype(self.d_type)

class ZeroMean(object):
    def __init__(self, mean, d_type=np.float32):
        self.mean = np.array(mean, dtype=d_type)

    def __call__(self, image, boxes=None, labels=None):
        image -= self.mean
        return image, boxes, labels

class Resize(object):
    def __init__(self, size=300):
        self.size = size

    def __call__(self, image, boxes=None, labels=None):
        image = cv2.resize(image, (self.size, self.size))
        return image, boxes, labels

class ToPercentCoords(object):
    def __call__(self, image, boxes=None, labels=None):
        height, width, channels = image.shape
        boxes[:, 0] /= width
        boxes[:, 2] /= width
        boxes[:, 1] /= height
        boxes[:, 3] /= height

        return image, boxes, labels

class Augmentation(object):
    def __init__(self, mode='train', size=300, mean=(104, 117, 123)):
        self.mean = mean
        self.size = size
        self.mode = mode

        if self.mode == 'train':
            self.augment = Compose([
                ToFloat(),    
                ToPercentCoords(),
                Resize(self.size),
                ZeroMean(self.mean)
            ])

        elif self.mode == 'test':
            self.augment = Compose([
                ToFloat(),
                Resize(self.size),
                ZeroMean(self.mean)
            ])

        else:
            raise Exception('Invalid mode!')

    def __call__(self, img, boxes, labels):
        return self.augment(img, boxes, labels)

ssd_train_augmentation = Augmentation(mode='train')
ssd_test_augmentation = Augmentation(mode='test')

In [ ]:
# taking from https://github.com/fchollet/keras/issues/1638
import threading


class threadsafe_iterator:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, iterator):
        self.iterator = iterator
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return next(self.iterator)


def threadsafe_generator(generator):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def wrapped_generator(*args, **kwargs):
        return threadsafe_iterator(generator(*args, **kwargs))
    return wrapped_generator

In [ ]:
class DataGenerator(object):
    def __init__(self, data, prior_boxes, mode='train', transform=None, shuffle=True,
                 batch_size=2, num_classes=3, box_scale_factors=[.1, .1, .2, .2]):
        self.data = data
        self.num_classes = num_classes
        self.prior_boxes = prior_boxes
        self.transform = transform
        self.box_scale_factors = box_scale_factors
        self.batch_size = batch_size
        self.mode = mode
        self.shuffle = shuffle

    @threadsafe_generator
    def flow(self):       
        keys = list(self.data.keys())
        if self.shuffle == True:
            random.shuffle(keys)

        while True:
            inputs = []
            targets = []
            for img in keys:
                img_data = cv2.imread(img).copy()
                box_data = self.data[img].copy()

                data = (img_data, box_data[:, :4], box_data[:, 4:])
                img_data, box_corners, labels = self.transform(*data)
                box_data = np.concatenate([box_corners, labels], axis=1)

#                 box_data = match(self.prior_boxes, box_data,
#                                               self.num_classes,
#                                               self.box_scale_factors)

                inputs.append(img_data)
                targets.append(box_data)
                if len(targets) == self.batch_size:
                    inputs = np.asarray(inputs)
                    targets = np.asarray(targets)
                    yield (inputs, targets)
                    inputs = []
                    targets = []

train_data_gen = DataGenerator(data=train_data, prior_boxes=precompute_priors(point_form=True), shuffle=False,
                               transform=ssd_train_augmentation, batch_size=1)

batch = train_data_gen.flow()

In [ ]:
# hyper-parameters
batch_size = 2
num_epochs = 250
alpha_loss = 1.0
learning_rate = 1e-3
weight_decay = 5e-4
gamma_decay = 0.1
negative_positive_ratio = 3
num_classes = 3
base_weights_path = 'weights_vgg.hdf5'

In [ ]:
model = SSD(n_classes=num_classes)
model.load_weights(base_weights_path, by_name=True)
prior_boxes = precompute_priors(point_form=True)
criterion = MultiboxLoss(num_classes, negative_positive_ratio, alpha_loss)
optimizer = Adam(lr=learning_rate, decay=weight_decay)
model.compile(optimizer, loss=criterion)
train_data_generator = DataGenerator(data=train_data, prior_boxes=precompute_priors(point_form=True),
                                     mode='train', transform=ssd_train_augmentation,
                                     batch_size=batch_size, num_classes=num_classes)

test_data_generator = DataGenerator(data=test_data, prior_boxes=precompute_priors(point_form=True),
                                     mode='test', transform=ssd_test_augmentation,
                                     batch_size=batch_size, num_classes=num_classes)

In [ ]:
import os


model_name = 'ssd300'
model_path = 'trained_models/' + model_name + '/'
save_path = model_path + 'weights.{epoch:02d}-{val_loss:.2f}.hdf5'
if not os.path.exists(model_path):
    os.makedirs(model_path)
log = CSVLogger(model_path + model_name + '.log')
checkpoint = ModelCheckpoint(save_path, verbose=1, save_weights_only=True)
# reduce_on_plateau = ReduceLROnPlateau(factor=gamma_decay, verbose=1)
# scheduler = LearningRateManager(learning_rate, gamma_decay, scheduled_epochs)
# learning_rate_schedule = LearningRateScheduler(scheduler, verbose=1)
callbacks = [checkpoint, log]
# callbacks = [checkpoint, log, reduce_on_plateau]

In [17]:
model.fit_generator(train_data_generator.flow(),
                    steps_per_epoch=int(len(train_data) / batch_size),
                    epochs=num_epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=test_data_generator.flow(),
                    validation_steps=int(len(test_data) / batch_size),
                    use_multiprocessing=False,
                    workers=1)



Epoch 1/250


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in log


158/158 [==============================] - 21s 131ms/step - loss: 1047.0767 - val_loss: 2.7823

Epoch 00001: saving model to trained_models/ssd300/weights.01-2.78.hdf5
Epoch 2/250
158/158 [==============================] - 14s 91ms/step - loss: 6.0180 - val_loss: 2.5389

Epoch 00002: saving model to trained_models/ssd300/weights.02-2.54.hdf5
Epoch 3/250
158/158 [==============================] - 15s 92ms/step - loss: 5.7530 - val_loss: 2.4015

Epoch 00003: saving model to trained_models/ssd300/weights.03-2.40.hdf5
Epoch 4/250
158/158 [==============================] - 14s 91ms/step - loss: 5.5912 - val_loss: 2.3044

Epoch 00004: saving model to trained_models/ssd300/weights.04-2.30.hdf5
Epoch 5/250
158/158 [==============================] - 14s 91ms/step - loss: 5.4758 - val_loss: 2.2285

Epoch 00005: saving model to trained_models/ssd300/weights.05-2.23.hdf5
Epoch 6/250
158/158 [==============================] - 14s 91ms/step - loss: 5.4079 - val_loss: 2.1692

Epoch 00006: saving mode

In [21]:
# from google.colab import drive
# drive.mount('/content/drive')
# !git clone https://gist.github.com/dc7e60aa487430ea704a8cb3f2c5d6a6.git /tmp/colab_util_repo
# !mv /tmp/colab_util_repo/colab_util.py colab_util.py 
# !rm -r /tmp/colab_util_repo
# from colab_util import *
# drive_handler = GoogleDriveHandler()
# drive_handler.upload('trained_models/ssd300/weights.250-0.22.hdf5', parent_path='IUST-DIP')

'1qplmVDF16ahIprrl2sPK9sAd8p1TnUjQ'

## README

ALL parts except following ones has been implemented:
1. Match function: to match precomputed prior boxes with the ones model has predicted
2. NMS: Non-max suppression for prediciton
3. Negative Mining: Reducing number of negative samples w.r.t. positive ones

Parts Have been IMPLEMENTED:
1. Model definition
2. Loss function
3. Data Generator
4. Augmentation
5. Transforms
6. Prior Box computations

I will finish it after final exams, but at the time of deadline, I had many other daily life (SURVIVAL) problems to deal.
Thank you for your time.